# **Basics of ABC testing for Mean and Proportions**

In [10]:
import numpy as np
import pandas as pd
import statsmodels
import pingouin
from scipy import stats

## **ABC Test with Proportion metrics**

When doing ABC testing with proportion metrics, one workflow that can be used is as follows:
1. **Do a Chi-square test for Independence** <br/>
   This is to test whether there is significant difference between group proportions.
2. **If the result rejects H0:**<br/>
   Compare every possible pair of groups via Z-proportion test. **Adjust p-value for each comparison to maintain overall statistical strength.**

### **Why Adjust the p-value?**

Since we are doing multiple pairwise comparison, by assuming independence, the probability of obtaining at least one "Reject H0" conclusion by pure randomness is $(1-\alpha)^6$. 

For $\alpha=0.05$ that is 0.265. Therefore, we can observe an explosion of Type I error chance of occuring in the set of comparison. 

To adjust p-value, we must base $\alpha$ on $\alpha^*$, $\alpha^*$ definition varies between methods, but usuall it defines significance level that projects the error rate of the entire sets of comparison.

#### **Method 1: Bonferroni-correction**
Here we define $\alpha^*$ as $Pr(False\ Positive\ge1)$ or Family Wise Error Rate (FWER), which in this case 0.05. This is done by simply dividing $\alpha$ with n number of comparison. If we have 3 then $\alpha$ for each fomparison is 0.0167, meaning that in each comparison it needs p-value < 0.0167 to be able to reject H0.
$$
\begin{align*}
  (1-\alpha)^n &= 1-\alpha^*\\
  \alpha &= 1-(1-\alpha^*)^{1/n} \le \alpha^*/n
\end{align*}
$$

**Weakness:**
Too conservative, therefore significantly increases the Type II error (False Negative) rate.

#### **Method 2: Benjamini-Hochberg-correction**
BH-correction control the False Discovery Rate (FDR) instead of FWER. In this approach, we set $\alpha^*$ as the expected FDR threshold.
$$
\begin{align*}
  \mathbb{E}(\frac{False\ Positive}{False\ Positive + True\ Positive}) \le FDR = \alpha^*
\end{align*}
$$
Therefore, $\alpha^*$ now means that for n pairs of comparison,  we're willing to accept up to $\alpha^*$% of significant results being false positives.
BH-correction follows this steps:
* Get the original p-value
* Sort pair comparison by p-value ascending
* Rank each comparison with k=1 for min(p-value) and k=n for max(p-value)
* Calculate adj-p-value with:<br/>
    adj-p-value $= n/k \cdot$ p-value

**Which approach to use?**

Simply: If we can't have any false positive, then choose to control FWER, otherwise usually pick FDR control since FWER can yield non-meaninful results.

In [2]:
# suppose we have the following experiment data
experiment_group = ['control','variant1','variant2','variant3']
count = np.array([1062, 825, 1289, 1228])
nobs = np.array([8333, 8002, 8251, 8275])

abc_prop_df = pd.DataFrame({
    'experiment_group': experiment_group,
    'event': count,
    'sample': nobs,
    'proportion': (count/nobs).round(4)
})
abc_prop_df

,experiment_group,event,sample,proportion
0,control,1062,8333,0.1274
1,variant1,825,8002,0.1031
2,variant2,1289,8251,0.1562
3,variant3,1228,8275,0.1484


To determines the clear winner of variants (if any), we can formulate the ABC testing results using the following program:

In [3]:
# because chi-square test is significant, we continue with t-tests with adjusted p-values
from statsmodels.stats.proportion import proportions_chisquare
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.proportion import proportions_ztest
from itertools import combinations

class ABC_TestProp:
    def __init__(self, experiment_df: pd.DataFrame|None, alpha: float=0.05,
                 sample_metric: str='sample', event_metric: str='event', prop_metric: str=None):
        self.experiment_df = experiment_df.copy().reset_index(drop=True)
        self.sample_metric = sample_metric
        self.event_metric  = event_metric
        self.alpha = alpha
        if prop_metric is None:
            self.experiment_df['proportion'] = self.experiment_df[event_metric]/self.experiment_df[sample_metric]
        else:
            self.experiment_df.rename(columns={prop_metric:'proportion'}, inplace=True)
    
    def evaluate(self):
        event_count = self.experiment_df[self.event_metric].values
        n_obs = self.experiment_df[self.sample_metric].values
        chi_p_val = proportions_chisquare(event_count, n_obs)[1]
        
        if chi_p_val < self.alpha:
            print(f"Significant result from chi-square test with p-value = {chi_p_val:.6f}")
            return self.posthoc_ztest()
        else:
            print(f"Chi-square test for independence is not significant (p-value = {chi_p_val:.5f})")
    
    def posthoc_ztest(self):
      pairs = list(combinations([i for i in range(len(self.experiment_df))], 2))
      z_crit = stats.norm.ppf(1-self.alpha/2)
      col_list = ['pair', 'prop_left', 'prop_right', 'prop_diff', 'p_value', 'adj_p_value', 
                  'prop_diff_low_ci', 'prop_diff_high_ci']
      # Empty dataframe to present results
      posthoc_df = pd.DataFrame({col:[] for col in col_list})
      
      for pair in pairs:
          pair = list(pair)
          # Extract pair data
          pair_data = self.experiment_df.loc[pair, ['experiment_group', 'sample', 'event', 'proportion']]
          pair_name = pair_data['experiment_group'].str.cat(sep=' v.s. ')
          pair_prop = list(pair_data['proportion'].values)
          prop_diff = pair_prop[0] - pair_prop[1]
          
          # Calculate p-value between pairs
          pair_event_count = pair_data['event']
          pair_n_obs       = pair_data['sample']
          pair_p_val = proportions_ztest(pair_event_count, pair_n_obs, alternative='two-sided')[1]
          
          # Calculate CI of difference in proportion between pair
          p1, p2 = pair_data['proportion']
          n1, n2 = pair_data['sample']
          p_pooled = pair_data['event'].sum()/pair_data['sample'].sum()
          pair_SE_pooled = np.sqrt(p_pooled*(1-p_pooled)*(1/n1 + 1/n2))
          pair_CI_lower = prop_diff - z_crit*pair_SE_pooled
          pair_CI_upper = prop_diff + z_crit*pair_SE_pooled
          
          # Append z-test result to posthoc_df
          val_list = [pair_name]+pair_prop+[prop_diff, pair_p_val, None, pair_CI_lower, pair_CI_upper]
          posthoc_df.loc[len(posthoc_df)] = val_list
        
      # p-value correction
      is_signif, adj_p_value = multipletests(posthoc_df['p_value'], alpha=self.alpha, method='fdr_bh')[0:2]
      posthoc_df['adj_p_value'] = adj_p_value

      return posthoc_df

In [4]:
abc_test_example = ABC_TestProp(abc_prop_df, alpha=0.05, prop_metric='')
abc_test_example.evaluate()

Significant result from chi-square test with p-value = 0.000000


,pair,prop_left,prop_right,prop_diff,p_value,adj_p_value,prop_diff_low_ci,prop_diff_high_ci
0,control v.s. variant1,0.1274,0.1031,0.0243,1.137315e-06,1.705972e-06,0.014494,0.034106
1,control v.s. variant2,0.1274,0.1562,-0.0288,1.081921e-07,2.163842e-07,-0.039418,-0.018182
2,control v.s. variant3,0.1274,0.1484,-0.0210,9.002234e-05,1.080268e-04,-0.031487,-0.010513
3,variant1 v.s. variant2,0.1031,0.1562,-0.0531,7.820956e-24,4.692574e-23,-0.063444,-0.042756
4,variant1 v.s. variant3,0.1031,0.1484,-0.0453,3.237856e-18,9.713568e-18,-0.055502,-0.035098
5,variant2 v.s. variant3,0.1562,0.1484,0.0078,1.615949e-01,1.615949e-01,-0.003157,0.018757


**Results:**
1. All pairs have statistically difference in proportions, except for variant2 and variant3
2. Variant1 is worse than control (p-value* < 0.05)
3. Our overall winners are both Variant2 and Variant3. As they are sigificantly better than control (p-value* < 0.05).

## **ABC Test with Mean metrics**

Similar to doing ABC testing with proportion metrics, the workflow to evaluate the results consist of evaluating groupwise difference, and then pairwise comparison if significant:
1. **Do a one-way ANOVA** <br/>
   This is to test whether there is significant difference between group mean.
2. **If the result rejects H0:**<br/>
   Compare every possible pair of groups via Tukey HSD post-hoc test

### **About Tukey HSD**
It is a more conservative way of doing t-test, therefore usually we don't need to adjust the p-value from it since we already need greater evidence of rejecting H0.

Same with Bonferroni-correction, it also controls the **family-wise error rate across all the pairs being compared, altough with different methods**

In [6]:
# data snippet
abc_transact_df = pd.read_csv('datasets/abc_transact_df.csv')
abc_transact_df.head()

,control,variant1,variant2
0,102400,184899,148000
1,76600,213500,119600
2,152200,114300,279500
3,99000,189700,105800
4,132600,164800,133300


In [7]:
# summary statistics
abc_transact_df.describe()

,control,variant1,variant2
count,200.000000,200.000000,200.000000
mean,125963.445000,138620.420000,144345.480000
std,41161.616418,49525.589042,49648.370821
min,13400.000000,-18000.000000,400.000000
25%,99225.000000,110274.250000,110525.000000
50%,130700.000000,137300.000000,142700.000000
75%,156250.000000,173200.000000,175175.000000
max,244800.000000,275300.000000,317800.000000


In [8]:
# Preprocess data -> tidy data
abc_transact_clean = abc_transact_df.melt(
    id_vars=None,
    var_name = 'experiment_group',
    value_name='trx_amount'
)
abc_transact_clean.head()

,experiment_group,trx_amount
0,control,102400
1,control,76600
2,control,152200
3,control,99000
4,control,132600


In [11]:
# step 1: one-way ANOVA
anova_res = abc_transact_clean.anova(dv='trx_amount', between='experiment_group', detailed=True)
anova_res.loc[:,['Source','SS','DF','MS','F','p-unc']]

,Source,SS,DF,MS,F,p-unc
0,experiment_group,3.539164e+10,2,1.769582e+10,8.028927,0.000362
1,Within,1.315793e+12,597,2.204008e+09,NaN,NaN


In [15]:
# After we found out that ANOVA result is significant
# We can continue with Tukey post-hoc test
z_crit = stats.norm.ppf(1-0.05/2)

tukey_res = abc_transact_clean.pairwise_tukey(dv='trx_amount', between='experiment_group').round(3)
tukey_res['delta'] = tukey_res['mean(B)'] - tukey_res['mean(A)']
tukey_res['lower_bound'] = tukey_res['delta'] - z_crit * tukey_res['se']
tukey_res['upper_bound'] = tukey_res['delta'] + z_crit * tukey_res['se']
tukey_res

,A,B,mean(A),mean(B),diff,se,T,p-tukey,hedges,delta,lower_bound,upper_bound
0,control,variant1,125963.445,138620.42,-12656.975,4694.686,-2.696,0.020,-0.277,12656.975,3455.559521,21858.390479
1,control,variant2,125963.445,144345.48,-18382.035,4694.686,-3.915,0.000,-0.402,18382.035,9180.619521,27583.450479
2,variant1,variant2,138620.420,144345.48,-5725.060,4694.686,-1.219,0.442,-0.115,5725.060,-3476.355479,14926.475479


**Results:**

1. Variant1 and Variant2 together are winner of this experiment, as they are better in delivering average of transaction amount than control (p-value < 0.05).
2. Variant1 and Variant2 have on-par average of transaction amount (p-value > 0.05).